Importing Required Libraries

In [0]:
# Import required libraries 

from torch.optim.lr_scheduler import _LRScheduler
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.autograd import Variable
from torch.optim import SGD
import torch.optim as optim
import torch.nn as nn
import torchvision
import numpy as np
import torch


Architecture in Class

In [0]:
cfg = {    
    'D' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']
}

class VGG16(nn.Module):
    def __init__(self, features, num_class=100):
        super().__init__()
        self.features = features

        self.classifier = nn.Sequential(
            nn.Linear(512, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_class)
        )

    def forward(self, X):
        op = self.features(X)
        op = op.view(op.size()[0], -1)
        op = self.classifier(op)
    
        return op

In [0]:
def create_layers(cfg, batch_norm=False):
    layers_list = []

    input_channel = 3
    for n in cfg:
        if n == 'M':
            layers_list += [nn.MaxPool2d(kernel_size=2, stride=2)]
            continue

        layers_list += [nn.Conv2d(input_channel, n, kernel_size=3, padding=1)]

        if batch_norm:
            layers_list += [nn.BatchNorm2d(n)]
        
        layers_list += [nn.ReLU(inplace=True)]
        input_channel = n
    
    return nn.Sequential(*layers_list)

In [0]:
def vgg16_bn():
    return VGG16(create_layers(cfg['D'], batch_norm=True))

Get CIFAR 100 dataset

In [0]:
def get_dataloader(batch_size=16, num_workers=2, shuffle=True):
    transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor()
    ])

    cifar100_training = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
    cifar100_training_loader = DataLoader(
        cifar100_training, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size)

    return cifar100_training_loader


Get CIFAR 100 test dataset

In [0]:
def get_test_dataloader(batch_size=16, num_workers=2, shuffle=True):
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    cifar100_test = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
    cifar100_test_loader = DataLoader(
        cifar100_test, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size)

    return cifar100_test_loader

In [31]:
train_loader = get_dataloader(num_workers=2, batch_size=128)

Files already downloaded and verified


In [32]:
test_loader = get_test_dataloader(num_workers=2, batch_size=128)

Files already downloaded and verified


In [0]:
vggnet = vgg16_bn()

In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [0]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    vggnet = vggnet.to(device)

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vggnet.parameters(), lr=0.001, momentum=0.9)

In [0]:
# Train model
for epochs in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        outputs = vggnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training!')

In [0]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [0]:
dataiter = iter(test_loader)
images, label = dataiter.next()
# print images
print('Ground Truth: ', ' '.join('%5s' % label[j] for j in range(4)))

GroundTruth:  tensor(55) tensor(19) tensor(68) tensor(16)


In [0]:
vggnet.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [0]:
output = vggnet(images.to(device))

In [0]:
_, predicted = torch.max(output, 1)
print('Predicted: ', (predicted[j] for j in range(4)))

Predicted:  <generator object <genexpr> at 0x7f7a74112ca8>


In [0]:
#Evaluating on test data
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        #images, labels = data
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on test images is {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 12 %
